<a href="https://colab.research.google.com/github/phionex2/Deep-Learning/blob/main/dgx_benchamrking_tf_mirrored_stratergy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="4"

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [4]:
!pip install tensorflow

In [5]:
import tensorflow as tf

print(tf.__version__)

2.15.0


In [6]:
tf.test.is_built_with_cuda()

True

In [7]:
(X_train, y_train), (X_test,y_test) = tf.keras.datasets.cifar10.load_data()

170498071/170498071 [==============================] - 2s 0us/step


In [8]:
X_train.shape

(50000, 32, 32, 3)

In [9]:
y_train.shape

(50000, 1)

In [10]:

classes = ["airplane","automobile","bird","cat","deer","dog","frog","horse","ship","truck"]

In [11]:
classes[y_train[3][0]]

'deer'

In [12]:
y_test.shape
X_train.shape

(50000, 32, 32, 3)

In [13]:
X_train_scaled = X_train / 255
X_test_scaled = X_test / 255

In [14]:
y_train_categorical = keras.utils.to_categorical(
    y_train, num_classes=10, dtype='float32'
)
y_test_categorical = keras.utils.to_categorical(
    y_test, num_classes=10, dtype='float32'
)

In [15]:
y_train[0:5]

array([[6],
       [9],
       [9],
       [4],
       [1]], dtype=uint8)

In [16]:
y_train_categorical[0:5]

array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [17]:
def get_model():
    model = keras.Sequential([
            keras.layers.Flatten(input_shape=(32,32,3)),
            keras.layers.Dense(3000, activation='relu'),
            keras.layers.Dense(1000, activation='relu'),
            keras.layers.Dense(10, activation='sigmoid')
        ])

    model.compile(optimizer='SGD',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model


In [18]:
train_tf_dataset = tf.data.Dataset.from_tensor_slices((X_train_scaled, y_train_categorical))
test_tf_dataset = tf.data.Dataset.from_tensor_slices((X_test_scaled, y_test_categorical))

In [19]:

strategy = tf.distribute.MirroredStrategy()

In [20]:
strategy.num_replicas_in_sync

1

In [21]:
BATCH_SIZE_PER_REPLICA = 250
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync


train_dataset = train_tf_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_tf_dataset.batch(BATCH_SIZE)

In [23]:

%%timeit -n1 -r1
with strategy.scope():
    gpu_model = get_model()
    gpu_model.fit(train_dataset, epochs=10)

Epoch 1/10
200/200 [==============================] - 53s 262ms/step - loss: 1.9765 - accuracy: 0.3020
Epoch 2/10
200/200 [==============================] - 54s 269ms/step - loss: 1.8160 - accuracy: 0.3649
Epoch 3/10
200/200 [==============================] - 53s 266ms/step - loss: 1.7562 - accuracy: 0.3874
Epoch 4/10
200/200 [==============================] - 52s 262ms/step - loss: 1.7138 - accuracy: 0.4020
Epoch 5/10
200/200 [==============================] - 53s 267ms/step - loss: 1.6796 - accuracy: 0.4140
Epoch 6/10
200/200 [==============================] - 52s 259ms/step - loss: 1.6504 - accuracy: 0.4244
Epoch 7/10
200/200 [==============================] - 51s 255ms/step - loss: 1.6249 - accuracy: 0.4345
Epoch 8/10
200/200 [==============================] - 52s 259ms/step - loss: 1.6021 - accuracy: 0.4432
Epoch 9/10
200/200 [==============================] - 51s 256ms/step - loss: 1.5817 - accuracy: 0.4501
Epoch 10/10
200/200 [==============================] - 53s 264ms/step - l

In [24]:
%%timeit -n1 -r1
with tf.device('/CPU:0'):
    cpu_model = get_model()
    cpu_model.fit(train_dataset, epochs=10)

Epoch 1/10
200/200 [==============================] - 54s 263ms/step - loss: 1.9782 - accuracy: 0.3002
Epoch 2/10
200/200 [==============================] - 52s 263ms/step - loss: 1.8134 - accuracy: 0.3672
Epoch 3/10
200/200 [==============================] - 52s 261ms/step - loss: 1.7532 - accuracy: 0.3893
Epoch 4/10
200/200 [==============================] - 54s 271ms/step - loss: 1.7116 - accuracy: 0.4039
Epoch 5/10
200/200 [==============================] - 53s 267ms/step - loss: 1.6783 - accuracy: 0.4157
Epoch 6/10
200/200 [==============================] - 53s 267ms/step - loss: 1.6497 - accuracy: 0.4261
Epoch 7/10
200/200 [==============================] - 53s 263ms/step - loss: 1.6249 - accuracy: 0.4348
Epoch 8/10
200/200 [==============================] - 52s 259ms/step - loss: 1.6029 - accuracy: 0.4431
Epoch 9/10
200/200 [==============================] - 52s 259ms/step - loss: 1.5830 - accuracy: 0.4501
Epoch 10/10
200/200 [==============================] - 51s 257ms/step - l